## Setup

In [1]:

#
import pandas as pd
import numpy as np


# make sql database connection with pyodbc
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')


SyntaxError: invalid syntax (1196091081.py, line 5)

# Air Quality

## Noise

In [ ]:
#Setup
import pandas as pd
import os from arcgis import GIS 
from arcgis.features import FeatureLayer
Import numpy as np
import arcpy

"F:\Research and Analysis\Noise\Monitoring\CNEL - Plan Areas\ALL_CNEL_PLANAREAS_RESULTS_(1991-2023).xlsx"

# Provide the path to your Excel file
excel_file_path = 'path/to/your/excel/file.xlsx'

# Read the Excel file into a DataFrame
Raw_df = pd.read_excel(excel_file_path)

# Define columns for the new DataFrame
columns_for_new_df = ['PAS_NAME','LAND USE', 'CNEL LAND USE','CNEL limit', ]

 #Create new dataframe using the defined columns
Noise23_df = pd.DataFrame(Raw_df, columns=columns_for_new_df)

#Reorganize to fit csv we will melt it into
Noise23_df(Description)= 'Average CNEL of all sites during timeframe'

Noise23_df(Year)=2023

#Field Mapping
field_mapping = {
    'Category', 'CNEL LAND USE '
    'Description': '',
    'Year': 'Year' 
    '2023 CNEL Average:  ,
    'Threshold Value': 'CNEL_Limit'}

# Rename fields based on field mappings--whateeer the df is
ready_df = filtered_df.rename(columns=field_mapping).drop(columns=[col for col in filtered_df.columns if col not in field_mapping])

#Export to CSV
ready_df.to_csv(os.path.join(working_folder, ThresholdNoise23.csv))

# Convert CSV to point feature class
arcpy.management.XYTableToPoint(csv_path, "oldStream_CSCI_Points", "LONGITUDE", "LATITUDE")

# Project to UTM Zone 10N
arcpy.Project_management("oldStream_CSCI_Points", "old_CSCI_Projected", arcpy.SpatialReference(26910))

print("Conversion and projection completed.")

## Stream/CSCI

In [1]:
#Setup
#Create Dictionary Usring Rest Service data

import pandas as pd
import os
from arcgis import GIS
from arcgis.features import FeatureLayer
import numpy as np
import requests
import arcpy

gis = GIS()
# Connect to TRPA Enterprise GIS Portal *if it's a service only shared with org
# portal_user = "TRPA_PORTAL_ADMIN"
# portal_pwd = str(os.environ.get('Password'))
# portal_url = "https://maps.trpa.org/portal/"
wk_memory = "memory" + "\\"

#Set up Workspace
# set workspace and sde connections 
working_folder = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries"
workspace      = "C:\GIS\Scratch.gdb"
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = r"C:\\GIS\\DB_CONNECT"


## CSV to be updated with new average
StreamAvg = os.path.join(working_folder,"ThresholdData_Stream.csv")


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

# Service URL
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'

# Get Stream data as a Spatially Enabled DataFrame
sdfStreamHab = get_fs_data(service_url)

# Keep only necessary columns
columnstokeep = ['SITE_NAME', 'STATION_TYPE', 'DURATION', 'LATITUDE', 'LONGITUDE', 'LTINFO', 'COUNT_VALUE', 'YEAR_OF_COUNT']
sdfStreamHab = sdfStreamHab.loc[:, columnstokeep]

# Query only 2020-2022 data
sdfStreamHab['Date'] = pd.to_datetime(sdfStreamHab['YEAR_OF_COUNT'], format='%Y')
filtered_df = sdfStreamHab[
    (sdfStreamHab['Date'].dt.year.between(2020, 2022)) & 
    (sdfStreamHab['DURATION'] == 'Long-term')
]

# Calculate the average of the 'COUNT_VALUE' column
AvgCSCI = filtered_df['COUNT_VALUE'].mean()

# Update CSV ThresholdData_Stream.csv
csv_path = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv"
existing_df = pd.read_csv(csv_path)

Description = "Average CSCI score of all trend sites (48 sites)"
Year = '2020-2022'
Value = AvgCSCI

existing_entry = existing_df[
    (existing_df['Description'] == Description) &
    (existing_df['Year'] == Year)
]

if existing_entry.empty:
    new_entry = {'Description': Description, 'Year': Year, 'Value': Value}
    existing_df = existing_df.append(new_entry, ignore_index=True)

existing_df.to_csv(csv_path, index=False)
print(f"CSV file '{csv_path}' has been updated.")



CSV file 'F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv' has been updated.


C:\Users\snewsome\AppData\Local\Temp\ipykernel_22588\3748317425.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_df = existing_df.append(new_entry, ignore_index=True)


: 

In [ ]:
#Use F:Research and Analysis/Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv' to update SDE.tabular.....table

In [ ]:
#Calculate Rating or CSCI values and then find the percent of total streams in each rating

#Calculate Rating for CSCI value
#Define a function to categorize CSCI values based on ranges
def categorize_value(value):
    if 0 <= value < 0.6:
        return 'poor'
    elif 0.6 <= value < 0.8:
        return 'marginal'
    elif 0.8 <= value < 1.0:
        return 'good'
    else:
        return 'excellent'

#Find average for each rating- Poor, Marginal, Good, Excellent